In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
示例：使用Stable Baselines3的SAC，并结合随机返回分解(RRD)来处理稀疏/延迟奖励环境。
代码仅用于演示核心思路，可能无法完全复现论文结果。
"""

import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from stable_baselines3 import SAC
from stable_baselines3.sac.policies import SACPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.logger import configure


# ========== 1. 环境包装：将每步奖励改为0，只在episode结束时返回总reward ==========
class EpisodicRewardWrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.cumulative_reward = 0.0

    def step(self, action):
        result = self.env.step(action)
        # 支持返回4个或5个值
        if len(result) == 5:
            obs, reward, terminated, truncated, info = result
            done = terminated or truncated
        else:
            obs, reward, done, info = result
        self.cumulative_reward += reward
        if done:
            reward = self.cumulative_reward
            self.cumulative_reward = 0.0
        else:
            reward = 0.0
        return obs, reward, done, info

    def reset(self, **kwargs):
        self.cumulative_reward = 0.0
        result = self.env.reset(**kwargs)
        # 支持返回tuple (obs, info) 或单个obs
        if isinstance(result, tuple):
            obs, _ = result
        else:
            obs = result
        return obs


# ========== 2. 轨迹存储（为了做 RRD） ==========
class TrajectoryReplay:
    """
    存储一条完整轨迹 (s, a, r, s_next, done)，
    其中 r 是“环境最终给出的总回报”（只有最后一步非0），其余为0。
    """
    def __init__(self):
        self.trajectories = []

    def add_trajectory(self, traj):
        """
        traj: list of (s, a, r, s_next, done)
        """
        self.trajectories.append(traj)

    def sample(self, batch_size):
        """
        随机采样batch_size条轨迹
        """
        indices = np.random.randint(0, len(self.trajectories), size=batch_size)
        return [self.trajectories[i] for i in indices]

    def __len__(self):
        return len(self.trajectories)


# ========== 3. 代理奖励模型： R(s,a) ==========
class RewardModel(nn.Module):
    """
    一个简单的两层网络，用于近似代理奖励 R(s,a)。
    """
    def __init__(self, state_dim, action_dim, hidden_size=256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim + action_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )

    def forward(self, s, a):
        x = torch.cat([s, a], dim=-1)
        return self.net(x)


# ========== 4. RRD训练函数：随机返回分解 ==========
def sample_subsequence_indices(traj_length, K):
    """
    从 [0, traj_length-1] 中随机选择 K 个不重复的索引
    """
    if K > traj_length:
        K = traj_length
    return np.random.choice(traj_length, size=K, replace=False)

def train_reward_model_rrd(reward_model, optimizer, trajectories, K=64, device='cpu'):
    """
    使用随机返回分解(RRD)来训练代理奖励模型。
    其中 trajectories 是一个batch的完整轨迹列表。
    """
    reward_model.train()
    M = len(trajectories)

    loss_sum = 0.0
    for traj in trajectories:
        # 轨迹的总回报 = 轨迹最后一步的reward(因为中间为0，最后一步为总和)
        R_ep = sum([t[2] for t in traj])  # 或者直接 traj[-1][2]

        # 收集这条轨迹的(s, a)
        s_list = []
        a_list = []
        for (s, a, r, s_next, done) in traj:
            s_list.append(s)
            a_list.append(a)

        states = torch.tensor(s_list, dtype=torch.float, device=device)
        actions = torch.tensor(a_list, dtype=torch.float, device=device)

        # 随机采样子序列
        Tj = len(traj)
        idx_subseq = sample_subsequence_indices(Tj, K)

        # 计算子序列上 R(s,a) 之和
        R_sa_sub = reward_model(states[idx_subseq], actions[idx_subseq])
        sum_R_sa_sub = R_sa_sub.sum()

        # (R_ep - sum_R_sa_sub)^2
        diff = (R_ep - sum_R_sa_sub) ** 2
        loss_sum += diff

    loss = loss_sum / M
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item()


# ========== 5. 收集轨迹的函数：使用当前策略进行若干回合采样 ==========
def collect_episodes(env, model, n_episodes, device='cpu'):
    trajectories = []
    for _ in range(n_episodes):
        result = env.reset()
        if isinstance(result, tuple):
            obs, _ = result
        else:
            obs = result
        done = False
        traj = []
        while not done:
            if obs is None or (hasattr(obs, '__len__') and len(obs) == 0):
                raise ValueError("Observation is empty, check env.reset() output")
            obs_tensor = torch.tensor(obs, dtype=torch.float, device=device).unsqueeze(0)
            with torch.no_grad():
                action, _states = model.predict(obs_tensor.cpu().numpy(), deterministic=False)
            result = env.step(action[0])
            if len(result) == 5:
                next_obs, reward, terminated, truncated, info = result
                done = terminated or truncated
            else:
                next_obs, reward, done, info = result
            traj.append((obs, action[0], reward, next_obs, done))
            obs = next_obs
        trajectories.append(traj)
    return trajectories


# ========== 6. 将RRD后的代理奖励存入SAC的ReplayBuffer ==========
def add_shaped_transitions_to_replay(model, reward_model, trajectories, gamma=0.99, device='cpu'):
    """
    对收集到的轨迹，用reward_model计算每步的代理奖励，然后存入SAC的replay buffer。
    """
    for traj in trajectories:
        states = []
        actions = []
        next_states = []
        dones = []
        for (s, a, r, s_next, d) in traj:
            states.append(s)
            actions.append(a)
            next_states.append(s_next)
            dones.append(d)

        # 转成张量
        s_tensor = torch.tensor(states, dtype=torch.float, device=device)
        a_tensor = torch.tensor(actions, dtype=torch.float, device=device)

        with torch.no_grad():
            shaped_rewards = reward_model(s_tensor, a_tensor).cpu().numpy().squeeze(-1)

        for i in range(len(traj)):
            s, a, _, s_next, d = traj[i]
            r_shaped = shaped_rewards[i]
            model.replay_buffer.add(
                s, s_next, a, r_shaped, d, infos=[{}],
            )




# ========== 7. 主函数：整合以上模块 ==========
def main():
    class Args:
        env = "Walker2d-v4"
        episodes = 2000
        steps_per_update = 1000
        rrd_k = 64
        device = "cuda"
    
    args = Args()

    device = torch.device(args.device)

    # 1. 创建环境 (只在最后一步返回总reward)
    base_env = gym.make(args.env)
    env = EpisodicRewardWrapper(base_env)
    # stable-baselines3 要求环境是VecEnv，故用 DummyVecEnv 包一下
    vec_env = DummyVecEnv([lambda: env])

    # 2. 创建SAC模型
    #    注意：此时环境返回的奖励是稀疏且延迟的，但我们并不直接用它来训练SAC。
    #    我们会手动往 replay_buffer 里塞“代理奖励”。
    model = SAC(
        policy="MlpPolicy",
        env=vec_env,  # 这里仍然要传env，但不会直接用它的奖励来学习
        verbose=1,
        seed=42,
        buffer_size=100000,
        learning_starts=0,       # 让它不必等待就能开始学习
        train_freq=1,            # 每次加一个transition就可以触发一次训练(后面我们自定义逻辑)
        gradient_steps=0,        # 我们会手动调用 train
        batch_size=256,
        gamma=0.99,
        device=device
    )

    model._logger = configure(folder=None, format_strings=["stdout"])

    # 3. 创建奖励模型
    #    需要知道 state_dim, action_dim
    state_dim = base_env.observation_space.shape[0]
    action_dim = base_env.action_space.shape[0]
    reward_model = RewardModel(state_dim, action_dim, hidden_size=256).to(device)
    reward_optimizer = optim.Adam(reward_model.parameters(), lr=3e-4)

    # 4. 收集轨迹 & RRD & SAC循环
    #    在每个回合：
    #      (a) 用当前策略收集一些episode
    #      (b) 用R(·)更新 reward_model (RRD)
    #      (c) 用更新后的 R(·) 生成 shaped reward，存入SAC的 replay buffer
    #      (d) 调用 model.train(gradient_steps=...) 进行SAC更新
    traj_replay = TrajectoryReplay()

    ep_record = []
    for ep in tqdm(range(args.episodes)):
        # (a) 收集1条或多条轨迹
        new_trajectories = collect_episodes(env, model, n_episodes=1, device=device)
        # 放入自定义的trajectory存储中
        for traj in new_trajectories:
            traj_replay.add_trajectory(traj)

        # (b) 如果轨迹数足够，就训练reward_model
        if len(traj_replay) >= 8:  # 大于一定数量再开始
            batch_size_trajectories = 8
            sampled_trajs = traj_replay.sample(batch_size_trajectories)
            rrd_loss = train_reward_model_rrd(
                reward_model, reward_optimizer, sampled_trajs,
                K=args.rrd_k, device=device
            )

        # (c) 将这些新轨迹用新的 reward_model 计算 shaped reward，添加到 SAC replay buffer
        add_shaped_transitions_to_replay(model, reward_model, new_trajectories, device=device)

        # (d) 调用 model.train(gradient_steps=...) 更新SAC
        #     这里的 steps-per-update 可视情况而定
        if len(traj_replay) >= 8:
            model.train(args.steps_per_update)

        # (e) 简单的评估：看一下最后一条trajectory的真实环境回报
        ep_return = sum([t[2] for t in new_trajectories[-1]])
        ep_record.append(ep_return)
        
        if ep % 10 == 0:
            print(f"Episode {ep+1}/{args.episodes}, real env return = {ep_return:.2f}")

    # 训练结束
    print("Training finished.")
    return ep_record

In [2]:
train1 = main()

Using cuda device


  0%|                                          | 5/2000 [00:00<00:40, 49.87it/s]

Episode 1/2000, real env return = 27.64


  0%|▏                                        | 10/2000 [00:08<34:55,  1.05s/it]

Episode 11/2000, real env return = -6.98


...

Episode 1981/2000, real env return = 3911.75


100%|████████████████████████████████████▊| 1991/2000 [5:23:17<01:33, 10.35s/it]

Episode 1991/2000, real env return = 4014.16


100%|█████████████████████████████████████| 2000/2000 [5:24:53<00:00,  9.75s/it]


Training finished.


In [3]:
np.save('rand_walker_train1.npy', train1)